# Setup
Connect to a Colab Runtime **with** GPU enabled! 
- Download files and libraries
- Download the model
- set up various environment variables
- create the database for the corpora of documents

In [ ]:
!git clone https://github.com/Mamiglia/privateGPT
!mv privateGPT/* ./
!pip install -r requirements.txt

Cloning into 'privateGPT'...
remote: Enumerating objects: 292, done.
remote: Counting objects: 100% (10/10), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 292 (delta 3), reused 7 (delta 3), pack-reused 282
Receiving objects: 100% (292/292), 105.71 KiB | 15.10 MiB/s, done.
Resolving deltas: 100% (142/142), done.
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 877.7/877.7 kB 51.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 329.1/329.1 kB 39.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.3/71.3 kB 11.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 76.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 123.2/123.2 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━

In [ ]:
# Download documents corpus
!rm -r source_documents/* privateGPT/
!gdown 1rkdQMKldyKYcebQrNcSzOzPUPPbHpXBL
!unzip documents.zip
!mv documents/* source_documents/
# ingest document corpus and create vectorial DB
!python ingest.py

In [ ]:
# Download GPT4ALL model
#!curl -O https://gpt4all.io/models/ggml-gpt4all-j-v1.3-groovy.bin

In [ ]:
# Set up Llama.cpp for using GPU
!pip uninstall -y llama-cpp-python
%env LLAMA_CUBLAS=1
%env CMAKE_ARGS=-DLLAMA_CUBLAS=on
%env FORCE_CMAKE=1
!pip install llama-cpp-python --no-cache-dir --verbose 

In [ ]:
from huggingface_hub import hf_hub_download
# Download Llama.cpp model
hf_hub_download(
    repo_id="vicuna/ggml-vicuna-7b-1.1", 
    filename="ggml-vic7b-q5_1.bin",
    local_dir='/content/'
)

In [ ]:
ENV = '''PERSIST_DIRECTORY=db
MODEL_TYPE=LlamaCpp
MODEL_PATH=/content/ggml-vic7b-q5_1.bin
EMBEDDINGS_MODEL_NAME=all-MiniLM-L6-v2
MODEL_N_CTX=1000
N_GPU_LAYERS=50
'''
with open('.env', 'w') as f:
  f.write(ENV)

# Run the model:
-M to mute the debug stdout

In [ ]:
!python privateGPT.py -M